In [1]:
""""
import torch
from torch import nn
#import torch.nn.functional as F
#import torch.optim as optim
#from torch.utils.data import Dataset, DataLoader
#import numpy as np
#from tqdm import tqdm, tqdm_notebook
"""""
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
##GPU 사용 시
device = torch.device("cuda:0")

In [5]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [6]:
import pandas as pd
df_data = pd.read_csv('C:/Users/rodls/OneDrive/바탕 화면/title.csv')

In [7]:
len(df_data)

20000

In [8]:
df_data.sample(n=10)

,V1,title,label
7757,7757,더벨2공장매각해유동성확보,1
1861,1861,이것은세일인가나눔인가급감성으로세대와교감하라,1
10120,10120,풀무원데이터기반의식품디지털클러스터구축,0
1581,1581,文대통령의료노조파업합의다행방역성과늘감사,2
19255,19255,필상생명과학인체미세전류활성화및생체조절장치특허기술개발,1
15828,15828,탈레반이15시간버스붙잡아탈진해서내리고버스서구타도,1
970,970,경남도밀양시한전등친환경산단건설가속,0
6714,6714,롯데홈쇼핑메타버스기업들손잡고가상쇼호스트선보인다,0
2817,2817,한국파마주가5경구용빈혈치료제아크루퍼국내독점공급계약,1
18118,18118,존리대표님삼성전자주식어떻게해야할까요허란의경제한끼,1


In [9]:
df_data.loc[(df_data['label'] == "하락"), 'label'] = 1  #하락 => -1
df_data.loc[(df_data['label'] == "중립"), 'label'] = 0  #중립 => 0
df_data.loc[(df_data['label'] == "상승"), 'label'] = 2  #상승 => 1

In [10]:
data_list = []
for q, label in zip(df_data['title'], df_data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [11]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[19999])
print(data_list[-1])

['웨버리오토골드만삭스가상승확신한다는10가지주식', '0']
['김해시팬데믹에도끄떡없는수출증가세작년동기대비228', '1']
['보수텃밭의민주당맨홍의락대구경제부시장사퇴', '1']
['페이코전자서명인증사업자선정민간인증서최초', '2']
['환경부中수리부수자원기술세미나첫개최물분야탄소중립이행협력강화', '1']
['환경부中수리부수자원기술세미나첫개최물분야탄소중립이행협력강화', '1']


In [12]:
#데이터 셋 test와 train으로 4:1분리
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [13]:
print(len(dataset_train))
print(len(dataset_test))

15000
5000


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5

In [16]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [17]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [18]:
data_train[0]

(array([   2,  726,  517, 7095, 7350, 7276, 6745, 6527, 7483, 6820, 5782,
        7044, 7911, 6290, 6312, 7609, 6122,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1]),
 array(18),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0)

In [19]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [22]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [23]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [24]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [25]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [26]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [27]:
train_dataloader

In [28]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

C:\Users\rodls\anaconda3\envs\bert\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/469 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.072227120399475 train acc 0.4375
epoch 1 batch id 201 loss 1.0592222213745117 train acc 0.40578358208955223
epoch 1 batch id 401 loss 0.9558568596839905 train acc 0.45012468827930174
epoch 1 train acc 0.46748400852878463


C:\Users\rodls\anaconda3\envs\bert\lib\site-packages\ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/157 [00:00<?, ?it/s]

...

epoch 14 test acc 0.9394904458598726


  0%|          | 0/469 [00:00<?, ?it/s]

epoch 15 batch id 1 loss 0.0005018952651880682 train acc 1.0
epoch 15 batch id 201 loss 0.0006507484940811992 train acc 0.9990671641791045
epoch 15 batch id 401 loss 0.00047592431656084955 train acc 0.9990648379052369
epoch 15 train acc 0.9991337953091685


  0%|          | 0/157 [00:00<?, ?it/s]

epoch 15 test acc 0.9392914012738853
